<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Метод-опорных-векторов" data-toc-modified-id="Метод-опорных-векторов-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Метод опорных векторов</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>LGBM</a></span></li><li><span><a href="#Выбор-модели" data-toc-modified-id="Выбор-модели-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Выбор модели</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Определение токсичных комментариев

**Иcходные данные**

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.

В нашем распоряжении набор данных с разметкой о токсичности правок.

**Цель исследования**

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо создать модель для классифицикации комментариев на позитивные и негативные.

Метрика качества *F1* на тестовой выборке должно быть не меньше 0.75.


**Ход исследования**

1. Обзор и подготовка данных.
2. Обучение и проверка различных моделей.
3. Финальное тестирование лучшей модели.
4. Вывод.

## Подготовка

### Импорт библиотек

In [44]:
import nltk
import numpy as np
import os
import pandas as pd
import re
import spacy
import warnings

from catboost import CatBoostClassifier

from lightgbm import LGBMClassifier

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [45]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Обзор данных

Импортируем данные.

In [2]:
pth1 = 'toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    df_toxic = pd.read_csv(pth1, index_col=[0])
elif os.path.exists(pth2):
    df_toxic = pd.read_csv(pth2, index_col=[0])
else:
    print('Файл не найден!')

Выполним первичный обзор данных.

In [3]:
print(df_toxic.info())
df_toxic.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


В таблице 159292 объекта. Индексы идут от 0 до 159450, то есть не все индексы совпадают с порядковыми номерами.

Согласно документации текст комментария находится в столбце `text`, а в столбце `toxic` — целевой признак.

Нам необходимо:
1. Посмотреть, что с индексами, проверить их на дублекаты.
2. Проверить данные на дублекаты.
3. Лемматизировать тексты.
4. Токенизировать тексты.

### Предобработка данных

Посмотрим, почему индексы не совпадают с порядковыми номерами.

In [4]:
df_toxic.index.duplicated().sum()

0

Проверим данные в целом на дублекаты.

In [5]:
df_toxic.duplicated().sum()

0

Дублекатов не обнаружено.

Приступим к лемматизации. Сначала очистим текст.

In [6]:
%%time

# 7 сек

def clear_text(text):
    clear = re.sub(r"[^a-zA-Z ']", " ", text)
    clear = ' '.join(clear.split())
    return clear

df_toxic['text'] = df_toxic['text'].apply(lambda x: clear_text(x))

CPU times: total: 2.14 s
Wall time: 2.14 s


Проверим результат.

In [7]:
df_toxic.head(15)

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D'aww He matches this background colour I'm se...,0
2,Hey man I'm really not trying to edit war It's...,0
3,More I can't make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0
5,Congratulations from me as well use the tools ...,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Лемматизируем.

In [8]:
%%time

#25 минут

nlp = spacy.load('en_core_web_sm')

df_toxic['lemma_text'] = df_toxic['text'].apply(lambda x: ' '.join([x.lemma_ for x in nlp(x)]))

CPU times: total: 27min 41s
Wall time: 27min 43s


Проверим результат.

In [9]:
df_toxic.head()

,text,toxic,lemma_text
0,Explanation Why the edits made under my userna...,0,Explanation why the edit make under my usernam...
1,D'aww He matches this background colour I'm se...,0,D'aww he match this background colour I be see...
2,Hey man I'm really not trying to edit war It's...,0,hey man I be really not try to edit war it be ...
3,More I can't make any real suggestions on impr...,0,More I can not make any real suggestion on imp...
4,You sir are my hero Any chance you remember wh...,0,you sir be my hero any chance you remember wha...


Разделим данные на выборки.

In [10]:
#выделим целевой признак

feature = df_toxic['lemma_text']
target = df_toxic['toxic']

In [11]:
#разделим данные на выборки

feature_train, feature_test, target_train, target_test = train_test_split(
    feature, target, test_size=0.2, stratify=target, random_state=42)


Приступим к токенизации.

In [12]:
def features_fit(data):
    nltk.download('stopwords')
    stop_words = set(nltk.corpus.stopwords.words('english'))
    corpus = data.values
    tfidf_vect = TfidfVectorizer(stop_words=stop_words).fit(corpus)
    return tfidf_vect

def features_transform(data, tfidf_vect):
    corpus = data.values
    tf_idf = tfidf_vect.transform(corpus)
    return tf_idf

In [48]:
tfidf_vect = features_fit(feature_train)
feature_train_tf_idf = features_transform(feature_train, tfidf_vect)
feature_test_tf_idf = features_transform(feature_test, tfidf_vect)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\1tech\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

Обучим несколько моделей-классификаторов и выберем лучшую:

1. Логистическая регрессия
2. Метод опорных векторов
3. Случайный лес
4. CatBoost
5. LGBM

### Логистическая регрессия 

In [46]:
%%time

# 5 минут

parametrs_lr ={'penalty': ['l1', 'l2', 'elasticnet', 'none'],
               'max_iter': range(50, 501, 100)
              }

gs_lr = GridSearchCV(LogisticRegression(random_state=42), 
                     parametrs_lr, 
                     scoring='f1', 
                     n_jobs=-1
                    )
gs_lr.fit(feature_train_tf_idf, target_train)

model_lr = gs_lr.best_estimator_
score_lr = gs_lr.best_score_
fit_time_lr = np.mean(gs_lr.cv_results_['mean_fit_time'])
score_time_lr = np.mean(gs_lr.cv_results_['mean_score_time'])

print(f'F1 модели Логистическая регрессия на валидационной выборке: {score_lr:.2f}')
print(f'Время обучения модели Логистическая регрессия: {fit_time_lr:.2f} c')
print(f'Время предсказания модели Логистическая регрессия: {score_time_lr:.2f} c')

F1 модели Логистическая регрессия на валидационной выборке: 0.76
Время обучения модели Логистическая регрессия: 19.84 c
Время предсказания модели Логистическая регрессия: 0.03 c
CPU times: total: 10.6 s
Wall time: 4min 41s


### Метод опорных векторов

In [28]:
%%time

# около 40 секунд

model_svc = None
score_svc = 0
best_kernel = None

for kernel in ['rbf', 'linear', 'poly']:
    model = SVC(random_state=42, kernel=kernel, max_iter=1000)
    scores = cross_validate(model, feature_train_tf_idf, target_train, scoring='f1', n_jobs=-1, cv=4)
    score = pd.Series(scores['test_score']).mean()
    if score > score_svc:
        score_svc = score
        model_svc = model
        best_kernel = kernel
        score_time_svc = pd.Series(scores['fit_time']).mean()
        fit_time_svc = pd.Series(scores['score_time']).mean()

print(f'F1 SVС на валидационной выборке: {score_svc:.2f}')
print(f'Параметр kernel: {best_kernel}')
print(f'Время обучения SVС: {fit_time_svc:.2f} c')
print(f'Время предсказания SVС: {score_time_svc:.2f} c')

RMSE RandomForestRegressor на валидационной выборке: 0.62
Параметр kernel: rbf
Время обучения RandomForestRegressor: 13.94 c
Время предсказания RandomForestRegressor: 62.39 c
CPU times: total: 297 ms
Wall time: 3min 42s


### Случайный лес

In [49]:
%%time

# примерно 2 минуты

model_rf = None
score_rf = 0
best_est = 0

for est in range(5, 61, 10):
    model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=5)
    scores = cross_validate(model, feature_train_tf_idf, target_train, scoring='f1', n_jobs=-1, cv=4)
    score = pd.Series(scores['test_score']).mean()
    if score > score_rf:
        score_rf = score
        model_rf = model
        best_est = est
        fit_time_rf = pd.Series(scores['fit_time']).mean()
        score_time_rf = pd.Series(scores['score_time']).mean()

print(f'F1 RandomForestClassifier на валидационной выборке: {score_rf:.2f}')
print(f'Параметр n_estimators: {best_est}')
print(f'Время обучения RandomForestClassifier: {fit_time_rf:.2f} c')
print(f'Время предсказания RandomForestClassifier: {score_time_rf:.2f} c')

F1 RandomForestClassifier на валидационной выборке: 0.00
Параметр n_estimators: 0
Время обучения RandomForestClassifier: 4.86 c
Время предсказания RandomForestClassifier: 0.11 c
CPU times: total: 1.36 s
Wall time: 1min 18s


### CatBoost

In [27]:
%%time 

# примерно 1 минуту

model_cbc = None
score_cbc = 0
best_lr = 0

for learning_rate in np.arange(0.1, 1.1, 0.1):
    model = CatBoostClassifier(random_state=42, 
                               verbose=False, 
                               iterations=12, 
                               learning_rate=learning_rate, 
                               depth=5, 
                               reg_lambda=0.2)
    scores = cross_validate(model, feature_train_tf_idf, target_train, scoring='f1', n_jobs=-1, cv=4)
    score = pd.Series(scores['test_score']).mean()
    if score > score_cbc:
        score_cbc = score
        model_cbc = model
        best_lr = learning_rate
        fit_time_cbc = pd.Series(scores['fit_time']).mean()
        score_time_cbc = pd.Series(scores['score_time']).mean()

print(f'F1 CatBoost на валидационной выборке: {score_cbc:.2f}')
print(f'Параметр n_estimators: {best_lr}')
print(f'Время обучения CatBoost: {fit_time_cbc:.2f} c')
print(f'Время предсказания CatBoost: {score_time_cbc:.2f} c')

RMSE RandomForestRegressor на валидационной выборке: 0.69
Параметр n_estimators: 1.0
Время обучения RandomForestRegressor: 71.43 c
Время предсказания RandomForestRegressor: 0.90 c
CPU times: total: 1.33 s
Wall time: 10min 22s


### LGBM

In [29]:
%%time 

# займет 1 мин

model_lgbm = None
score_lgbm = 0
best_lr = 0

for learning_rate in np.arange(0.1, 1.1, 0.1):
    model = LGBMClassifier(random_state=42, 
                            reg_alpha=0.2, 
                            reg_lambda=0.1, 
                            learning_rate=learning_rate)
    scores = cross_validate(model, feature_train_tf_idf, target_train, scoring='f1', n_jobs=-1, cv=4)
    score = pd.Series(scores['test_score']).mean()
    if score > score_cbc:
        score_lgbm = score
        model_lgbm = model
        best_lr = learning_rate
        fit_time_lgbm = pd.Series(scores['fit_time']).mean()
        score_time_lgbm = pd.Series(scores['score_time']).mean()

print(f'F1 LightGBM на валидационной выборке: {score_lgbm:.2f}')
print(f'Параметр n_estimators: {best_lr}')
print(f'Время обучения LightGBM: {fit_time_lgbm:.2f} c')
print(f'Время предсказания LightGBM: {score_time_lgbm:.2f} c')

F1 LightGBM на валидационной выборке: 0.70
Параметр n_estimators: 0.9
Время обучения LightGBM: 49.76 c
Время предсказания LightGBM: 1.03 c
CPU times: total: 1.22 s
Wall time: 10min 50s


### Выбор модели

Создадим таблицу значений.

In [50]:
sheet_models = pd.DataFrame({
        'Модель': ['Логистическая регрессия', 'Метод опорных векторов', 'Случайный лес', 'CatBoost', 'LGBM'],
        'F1': [score_lr, score_svc, score_rf, score_cbc, score_lgbm],
        'Время обучения': [fit_time_lr, fit_time_svc, fit_time_rf, fit_time_cbc, fit_time_lgbm],
        'Время предсказания': [score_time_lr, score_time_svc, score_time_rf, score_time_cbc, score_time_lgbm]
       })

display(sheet_models.reset_index(drop=True))

,Модель,F1,Время обучения,Время предсказания
0,Логистическая регрессия,0.761195,19.842389,0.025857
1,Метод опорных векторов,0.624674,13.939717,62.392377
2,Случайный лес,0.000000,4.855497,0.107462
3,CatBoost,0.692286,71.429488,0.897350
4,LGBM,0.701941,49.761919,1.032745


Из всех полученных моделей самую высокую точность имеет Логистическая регрессия. При этом она не является слишком долгой в обучении и самая быстрая по скорости предсказания.

## Тестирование

Рассичтаем F1 лучшей модели на тестовой выборке.

In [42]:
best_model_predict = model_lr.predict(feature_test_tf_idf)
best_model_score = f1_score(target_test, best_model_predict)
print(f'F1 лучшей модели на тестовой выборке: {best_model_score:.2f}')

F1 лучшей модели на тестовой выборке: 0.77


Сравним с моделью-пустышкой.

In [60]:
dummy_model = DummyClassifier()
dummy_model.fit(feature_train_tf_idf, target_train)
dummy_model_predict = dummy_model.predict(feature_test_tf_idf)
dummy_model_score = f1_score(target_test, dummy_model_predict)
print(f'F1 Dummy модели на тестовой выборке: {dummy_model_score:.2f}')

if dummy_model_score < best_model_score:
    print('Наша модель успешно прошла тестирование.')
else:
    print('Наша модель никуда не годится!')

F1 Dummy модели на тестовой выборке: 0.00
Наша модель успешно прошла тестирование.


## Выводы

Нам удалось создать модель опделяющую токсичность комметариев с F1 равным 0.77 на тестовой выборке.